In [ ]:
# Bootstrap (CI-safe)
# tags: [bootstrap]
import os, random, numpy as np
import ts2eg

SEED = 0
random.seed(SEED)
np.random.seed(SEED)

# Downshift knob: set TS2EG_CI=1 in CI to keep runtime small.
TS2EG_CI = os.getenv("TS2EG_CI") == "1"


# FIRM_DEMO: Capital–Labor Dynamics → Evolutionary Game Analysis (synthetic)

This notebook demonstrates a value‑neutral mapping of firm dynamics (capital–labor allocation)
to an evolutionary game using the `gamify-timeseries` scaffold.

We will:
1) Build **allocation shares** `X` over {LaborComp, CapEx, R&D, SG&A, Payouts}.
2) Define two payoffs: (A) **distributional surplus‑capture**; (B) **value‑gradient to future ROIC**.
3) Learn strategies via `nmf_on_X`, estimate `A` with `estimate_A_from_series`, search ESS via `find_ESS`.
4) Compute a falsifiable statistic for **systematic surplus capture** and test against a row‑shift null.

**Note:** Synthetic data; real analysis requires careful prep, rolling windows, nulls, and intervention checks.

In [ ]:
# --- Canonical imports (ts2eg only) ---
import ts2eg as gm
from ts2eg.core import nmf_on_X, growth_payoffs, estimate_A_from_series, find_ESS
try:
    from ts2eg import extensions as ext
except Exception:
    ext = None  # optional


## 1) Simulate a quarterly firm panel

In [ ]:
def simulate_firm(T=80, seed=0):
    rng = np.random.default_rng(seed)
    t = np.arange(T)
    VA = np.exp(0.01*t + 0.05*rng.standard_normal(T)).astype(float)  # value added
    lab_share = 0.58 + 0.05*np.sin(2*np.pi*t/24) + 0.03*rng.standard_normal(T)
    lab_share = np.clip(lab_share, 0.45, 0.70)
    cap_share = 1.0 - lab_share
    W = lab_share * VA                 # labor compensation
    P = cap_share * VA                 # operating surplus
    N = 5
    X = np.zeros((N, T))               # allocation shares
    X[0] = 0.45 + 0.05*np.sin(2*np.pi*(t+3)/20) + 0.02*rng.standard_normal(T)   # LaborComp
    X[1] = 0.20 + 0.03*np.sin(2*np.pi*(t+5)/16) + 0.02*rng.standard_normal(T)   # CapEx
    X[2] = 0.08 + 0.02*np.sin(2*np.pi*(t+8)/18) + 0.01*rng.standard_normal(T)   # R&D
    X[3] = 0.17 + 0.02*np.sin(2*np.pi*(t+2)/26) + 0.02*rng.standard_normal(T)   # SG&A
    X[4] = 0.10 + 0.02*np.sin(2*np.pi*(t+1)/22) + 0.02*rng.standard_normal(T)   # Payouts
    X = np.clip(X, 1e-6, None); X /= X.sum(axis=0, keepdims=True)
    roic = 0.08 + 0.01*np.sin(2*np.pi*t/12) \
           + 0.04*(0.3*X[2] + 0.2*X[1] - 0.1*X[4] - 0.05*X[3]) \
           + 0.01*rng.standard_normal(T)
    return X, VA, W, P, roic

X, VA, W, P, roic = simulate_firm(T=80, seed=2)
players = ['LaborComp','CapEx','R&D','SG&A','Payouts']
print('X shape:', X.shape)

## 2) Payoff A — distributional surplus‑capture proxy
We use Δlog of labor and capital shares of value‑added, assign to rows, and center across players.

In [ ]:
def dlog(arr):
    arr = np.asarray(arr, dtype=float)
    return np.diff(np.log(arr + 1e-12), prepend=np.log(arr[0] + 1e-12))

labor_share = W / (VA + 1e-12)
capital_share = P / (VA + 1e-12)
v_dist = np.zeros_like(X)
v_dist[0] = dlog(labor_share)
v_dist[4] = dlog(capital_share)

N = X.shape[0]
M_I = np.ones((N, N))/N; M_Z = np.eye(N) - M_I
vZ_dist = M_Z @ v_dist
print('v_dist shape:', v_dist.shape)

### Distributional test statistic and null
Statistic: \( D = \overline{v^{Z}_{\text{Payouts}}} - \overline{v^{Z}_{\text{Labor}}} \). Compare to a row‑shift null.

In [ ]:
def circular_shift_rows(M, seed=0):
    rng = np.random.default_rng(seed)
    N, T = M.shape
    Y = np.zeros_like(M)
    for i in range(N):
        k = int(rng.integers(0, T))
        Y[i] = np.roll(M[i], k)
    return Y

D = vZ_dist[4].mean() - vZ_dist[0].mean()
Ds = []
for b in range(500):
    vZ_null = M_Z @ circular_shift_rows(v_dist, seed=b)
    Ds.append(vZ_null[4].mean() - vZ_null[0].mean())
Ds = np.array(Ds)
z = (D - Ds.mean()) / (Ds.std() + 1e-12)
print('D (capital minus labor, mean Z‑payoff):', round(D, 5), '  z vs null:', round(z, 2))

## 3) Strategies and operator (distributional payoff)

In [ ]:
# Ensure strategy basis S exists before estimation
try:
    _ = S  # noqa: F821
except NameError:
    try:
        K = int(globals().get('K', 3))
    except Exception:
        K = 3
    S, H = nmf_on_X(X, k=K, iters=50, seed=1, normalize='l2')


In [ ]:
# Ensure strategy basis S exists before estimation
try:
    _ = S  # noqa: F821
except NameError:
    try:
        K = int(globals().get('K', 3))
    except Exception:
        K = 3
    S, H = nmf_on_X(X, k=K, iters=50, seed=1, normalize='l2')


In [ ]:
K = 3
S, H = nmf_on_X(X, k=K, iters=300, seed=1, normalize='l2')
est_dist = estimate_A_from_series(S, X, v_dist, k=K, ridge=1e-2)
A_dist = est_dist['A']
ess_dist = [r for r in find_ESS(A_dist, tol=1e-8, max_support=K) if r['is_ess']]
As, Aa = 0.5*(A_dist + A_dist.T), 0.5*(A_dist - A_dist.T)
print('R^2 (dist):', round(est_dist['R2'], 3), ' ESS:', len(ess_dist), '  skew ratio:', round(np.linalg.norm(Aa)/(np.linalg.norm(A_dist)+1e-12),3))

### Plot inferred strategy mixture `x(t)`

In [ ]:
plt.figure(figsize=(8,3))
for i in range(K):
    plt.plot(est_dist['Xk'][i], label=f'x_{i+1}')
plt.title('Strategy memberships over time'); plt.legend(); plt.show()

## 4) Payoff B — value‑gradient to future ROIC

In [ ]:
H = 4
J_future = np.roll(roic, -H)
v_val = value_gradient_payoffs(X, J_future, ridge=1e-2, standardize=True)
est_val = estimate_A_from_series(S, X, v_val, k=K, ridge=1e-2)
A_val = est_val['A']
ess_val = [r for r in find_ESS(A_val, tol=1e-8, max_support=K) if r['is_ess']]
Asv, Aav = 0.5*(A_val + A_val.T), 0.5*(A_val - A_val.T)
print('R^2 (value):', round(est_val['R2'], 3), ' ESS:', len(ess_val), '  skew ratio:', round(np.linalg.norm(Aav)/(np.linalg.norm(A_val)+1e-12),3))

## 5) Null test (row shifts) for value‑gradient fit

In [ ]:
def fit_R2_under_null(X, v, S, B=100):
    R2s = []
    for b in range(B):
        v_null = circular_shift_rows(v, seed=100+b)
        est = estimate_A_from_series(S, X, v_null, k=S.shape[1], ridge=1e-2)
        R2s.append(est['R2'])
    return np.array(R2s)

R2_null = fit_R2_under_null(X, v_val, S, B=100)
z_R2 = (est_val['R2'] - R2_null.mean()) / (R2_null.std() + 1e-12)
print('R^2 z‑score vs row‑shift null:', round(z_R2, 2))

## 6) Rolling‑window analysis (stability over time)

In [ ]:

# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

import numpy as np, matplotlib.pyplot as plt

def rolling_window_metrics(X, v, S, win=32, step=4, labor_idx=0, capital_idx=4):
    T = X.shape[1]
    out = {"t0":[], "t1":[], "R2":[], "skew_ratio":[], "D":[]}
    from ts2eg import estimate_A_from_series
    N = X.shape[0]
    M_I = np.ones((N,N))/N; M_Z = np.eye(N)-M_I
    for t0 in range(0, T-win+1, step):
        t1 = t0 + win
        est = estimate_A_from_series(S, X[:,t0:t1], v[:,t0:t1], k=S.shape[1], ridge=1e-2)
        A = est["A"]; As, Aa = 0.5*(A+A.T), 0.5*(A-A.T)
        skew_ratio = float(np.linalg.norm(Aa)/(np.linalg.norm(A)+1e-12))
        # D statistic if the payoff aligns with labor/capital rows; else NaN
        vZ = M_Z @ v[:,t0:t1]
        D = float(vZ[capital_idx].mean() - vZ[labor_idx].mean()) if v.shape[0]>max(labor_idx,capital_idx) else float("nan")
        out["t0"].append(t0); out["t1"].append(t1)
        out["R2"].append(est["R2"]); out["skew_ratio"].append(skew_ratio); out["D"].append(D)
    return out

# Compute rolling metrics for both payoffs
roll_dist = rolling_window_metrics(X, v_dist, S, win=32, step=4)
roll_val  = rolling_window_metrics(X, v_val,  S, win=32, step=4)

plt.figure(figsize=(10,4))
plt.plot(roll_dist["t1"], roll_dist["R2"], label="R2 (dist)")
plt.plot(roll_val["t1"],  roll_val["R2"],  label="R2 (value)")
plt.title("Rolling fit (R2)"); plt.legend(); plt.show()

plt.figure(figsize=(10,4))
plt.plot(roll_dist["t1"], roll_dist["skew_ratio"], label="skew ratio (dist)")
plt.plot(roll_val["t1"],  roll_val["skew_ratio"],  label="skew ratio (value)")
plt.title("Rolling skew ratio ||A_a||/||A||"); plt.legend(); plt.show()

plt.figure(figsize=(10,4))
plt.plot(roll_dist["t1"], roll_dist["D"], label="D (capital minus labor)")
plt.axhline(0, linestyle="--")
plt.title("Rolling surplus‑capture statistic D"); plt.legend(); plt.show()


## 7) Real‑data stub (bring your own 10‑Q/10‑K panel)

Expected CSV columns (quarterly): `date, LaborComp, CapEx, RnD, SGA, Payouts, VA, W, P, ROIC`.

Place a file at `data/firm_quarterly.csv`, or modify the path below. Shares `X` are built by column‑normalizing `{LaborComp, CapEx, RnD, SGA, Payouts}` per quarter. `VA` (value added) should satisfy approximately `VA ≈ W + P`.

In [ ]:

# --- Canonical imports with fallbacks ---
try:
    import ts2eg as gm
except Exception:
    try:
        from gamify_timeseries import *  # standalone canonical file
        import types as _types
        gm = _types.SimpleNamespace(
            helmert_Q=helmert_Q, projectors=projectors,
            static_game_from_series=static_game_from_series,
            var_information_sharing_game=var_information_sharing_game,
            nmf_on_X=nmf_on_X, estimate_A_from_series=estimate_A_from_series,
            find_ESS=find_ESS
        )
    except Exception:
        raise ImportError("Neither 'ts2eg' package nor 'gamify_timeseries.py' (standalone) is available.")
try:
    from ts2eg import extensions as ext
except Exception:
    try:
        import egt_extensions as ext  # last resort
    except Exception:
        ext = None  # not all notebooks need extensions

import os, pandas as pd, numpy as np
from ts2eg import nmf_on_X, value_gradient_payoffs, estimate_A_from_series, find_ESS

path = "data/firm_quarterly.csv"
if not os.path.exists(path):
    # Write a template to /mnt/data for convenience
    tmpl = pd.DataFrame({
        "date": pd.period_range("2010Q1", periods=12, freq="Q").astype(str),
        "LaborComp":  [45,46,47,48,47,46,45,44,45,46,46,47],
        "CapEx":      [20,21,22,21,22,23,22,21,20,20,21,22],
        "RnD":        [8,8,9,9,9,9,10,10,9,9,9,9],
        "SGA":        [17,16,16,16,16,16,16,17,17,17,17,17],
        "Payouts":    [10,9,8,8,8,8,7,8,9,8,7,5],
        "VA":         [100,102,104,103,105,106,108,109,110,112,113,115],
        "W":          [58,59,60,61,60,59,58,57,58,59,59,60],
        "P":          [42,43,44,42,45,47,50,52,52,53,54,55],
        "ROIC":       [0.08,0.081,0.082,0.081,0.083,0.084,0.085,0.084,0.083,0.082,0.082,0.083],
    })
    tmpl_path = "/mnt/data/firm_quarterly_template.csv"
    tmpl.to_csv(tmpl_path, index=False)
    print("Template written:", tmpl_path)
else:
    df = pd.read_csv(path)
    cols = ["LaborComp","CapEx","RnD","SGA","Payouts"]
    X_real = df[cols].to_numpy(dtype=float).T
    X_real = np.clip(X_real, 1e-9, None)
    X_real = X_real / (X_real.sum(axis=0, keepdims=True) + 1e-12)
    VA = df["VA"].to_numpy(float); W = df["W"].to_numpy(float); P = df["P"].to_numpy(float)
    ROIC = df.get("ROIC", pd.Series([np.nan]*len(df))).to_numpy(float)

    # Example payoff: value‑gradient to future ROIC (H=4 quarters lead)
    H = 4
    J_future = np.roll(ROIC, -H)
    v_real = value_gradient_payoffs(X_real, J_future, ridge=1e-2, standardize=True)

    # Strategies & operator
    S_r, H_r = nmf_on_X(X_real, k=3, iters=500, seed=2)
    est_r = estimate_A_from_series(S_r, X_real, v_real, k=3, ridge=1e-2)
    A_r = est_r["A"]
    ess_r = [r for r in find_ESS(A_r, tol=1e-8, max_support=3) if r["is_ess"]]
    print("Real‑data stub: R2=", round(est_r["R2"],3), "  ESS count=", len(ess_r))
